<a href="https://colab.research.google.com/github/klisman02/Data-Analysis-Project/blob/main/virtual_assistent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# assistant_colab.ipynb (coloque tudo numa célula do Colab)

# Instala dependências
!pip install SpeechRecognition gTTS wikipedia pyttsx3 playsound==1.2.2

import os
import wikipedia
import webbrowser
import speech_recognition as sr
from gtts import gTTS
from IPython.display import Audio, display

# Configura Wikipédia em português
wikipedia.set_lang("pt")

# --- Funções TTS ---
def speak(text, filename="output.mp3"):
    print("Assistente:", text)
    tts = gTTS(text=text, lang="pt")
    tts.save(filename)
    display(Audio(filename, autoplay=True))

# --- Funções STT (no Colab: só funciona com upload de arquivo) ---
def transcribe_audio(file_path):
    recognizer = sr.Recognizer()
    with sr.AudioFile(file_path) as source:
        audio = recognizer.record(source)
    try:
        text = recognizer.recognize_google(audio, language="pt-BR")
        print("Você disse:", text)
        return text.lower()
    except sr.UnknownValueError:
        return ""
    except sr.RequestError:
        speak("Erro ao acessar o serviço de reconhecimento.")
        return ""

# --- Intent simples ---
def handle_intent(text):
    if not text:
        return "unknown", text
    if "wikip" in text or "pesquisar" in text:
        return "wikipedia", text
    if "youtube" in text:
        return "youtube", text
    if "farmácia" in text or "farmacia" in text:
        return "pharmacy", text
    if "oi" in text or "olá" in text:
        return "greeting", text
    if "sair" in text or "tchau" in text:
        return "exit", text
    return "unknown", text

# --- Executor ---
def execute(intent, original_text):
    if intent == "wikipedia":
        term = original_text.replace("pesquisar", "").replace("na wikipedia","").strip()
        if not term:
            speak("Sobre o que você quer pesquisar na Wikipédia?")
            return
        try:
            summary = wikipedia.summary(term, sentences=2)
            speak(f"Segundo a Wikipédia: {summary}")
        except:
            speak("Não encontrei resultados na Wikipédia.")
    elif intent == "youtube":
        query = original_text.replace("youtube", "").strip()
        url = "https://www.youtube.com" if not query else f"https://www.youtube.com/results?search_query={query.replace(' ','+')}"
        speak("Abrindo YouTube...")
        print("URL:", url)
    elif intent == "pharmacy":
        url = "https://www.google.com/maps/search/farmácia+mais+próxima"
        speak("Mostrando farmácias próximas...")
        print("URL:", url)
    elif intent == "greeting":
        speak("Olá! Em que posso ajudar?")
    elif intent == "exit":
        speak("Até logo!")
        return "exit"
    else:
        speak("Desculpe, não entendi o comando.")

# --- Demonstração no Colab ---
# Você pode digitar um comando de texto para simular a voz:
while True:
    comando = input("\nDigite um comando (ou 'sair' para encerrar): ")
    intent, orig = handle_intent(comando.lower())
    res = execute(intent, orig)
    if res == "exit":
        break
